In [1]:
import os

In [2]:
import torch.backends.cudnn as cudnn
import torch
import yaml
from utils import AttrDict
import pandas as pd
import importlib
from train import train

In [3]:
# !pip install torch
# !pip install natsort
# !pip install torchvision
# !pip install pyyaml
# !pip install pandas
# !pip install nltk

In [4]:
cudnn.benchmark = True
cudnn.deterministic = False

In [5]:
torch.cuda.is_available()

True

In [6]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

In [7]:
opt = get_config("config_files/star-net.yaml")
train(opt, amp=False)
%tb

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: all_data/training
opt.select_data: ['']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_log dataset_root:    all_data/training	 dataset: 
all_data/training/training
sub-directory:	/training	 num samples: 2700
num total samples of : 2700 x 1.0 (total_data_usage_ratio) = 2700
num samples of  per batch: 16 x 1.0 (batch_ratio) = 16
--------------------------------------------------------------------------------
Total_batch_size: 16 = 16
--------------------------------------------------------------------------------
dataset_log dataset_root:    all_data/validation	 dataset: /
all_data/validation/validation
sub-directory:	/validation	 num samples: 300
-------------------------------------------

c:\Users\PIYAWAT\miniconda3\lib\site-packages\torch\nn\functional.py:4193: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


training time:  12.920438528060913
[10/2000] Train loss: 49.03160, Valid loss: 6.42916, Elapsed_time: 12.92144
Current_accuracy : 0.000, Current_norm_ED  : 0.0000
Best_accuracy    : 0.000, Best_norm_ED     : 0.0000
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
StoreTM                   |                           | 0.0000	False
Ambassadeur               |                           | 0.0000	False
--------------------------------------------------------------------------------
validation time:  13.329115629196167
training time:  8.07842206954956
[20/2000] Train loss: 6.70461, Valid loss: 5.48223, Elapsed_time: 34.32897
Current_accuracy : 0.000, Current_norm_ED  : 0.0000
Best_accuracy    : 0.000, Best_norm_ED     : 0.0000
----------------------------------------------------------------------------

SystemExit: 

c:\Users\PIYAWAT\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
